# MNIST training

## Data loading

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

INVERT = False

(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

ds_train

## Training dataset setup

In [ ]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img)

if INVERT:
    def invert(image, label):
        return (tf.cast(image, tf.float32) * -1.0) + 1.0, label
    inverted = ds_train.map(invert)
    ds_train = ds_train.concatenate(inverted)

ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples*(INVERT+1))
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

## Testing dataset setup

In [ ]:
ds_test = ds_test.map(normalize_img)

if INVERT:
    inverted = ds_test.map(invert)
    ds_test = ds_test.concatenate(inverted)

ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

## Model creation

In [ ]:
max_pool = tf.keras.layers.MaxPool2D((2, 2), (2, 2), padding='same')

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(
        filters=16,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu),
    max_pool,
    tf.keras.layers.Conv2D(
        filters=32,
        kernel_size=5,
        padding="same",
        activation=tf.nn.relu),
    max_pool,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

## Model fitting

In [ ]:
model.fit(
    ds_train,
    epochs=1,
    validation_data=ds_test,
)

## Prediction image loading

In [ ]:
!wget -q https://github.com/milliams/machine_learning/archive/master.zip
!unzip -q -j -o master.zip

In [ ]:
import numpy as np
from skimage.io import imread

images = []
for i in list(range(1,10)) + ["dog"]:
    images.append(np.array(imread(f"{i}.png")/255.0, dtype='float32'))
images = np.array(images)[:,:,:,np.newaxis]
test_data = tf.convert_to_tensor(images)

## Make the predictions

In [ ]:
probabilities = model.predict(test_data)

## Output stats on the predictions

In [ ]:
truths = list(range(1, 10)) + ["dog"]

table = []
for truth, probs in zip(truths, probabilities):
    prediction = probs.argmax()
    if truth == 'dog':
        print(f"{truth}. CNN thinks it's a {prediction} ({probs[prediction]*100:.1f}%)")
    else:
        print(f"{truth} at {probs[truth]*100:4.1f}%. CNN thinks it's a {prediction} ({probs[prediction]*100:4.1f}%)")
    table.append((truth, probs))

In [ ]:
def print_table(table):
    print("""<table cellpadding="0" style="border-collapse: collapse; border-style: hidden;">
    <thead>
    <tr>
    <td><b>Image</b></td>
    <td><b>0</b></td>
    <td><b>1</b></td>
    <td><b>2</b></td>
    <td><b>3</b></td>
    <td><b>4</b></td>
    <td><b>5</b></td>
    <td><b>6</b></td>
    <td><b>7</b></td>
    <td><b>8</b></td>
    <td><b>9</b></td>
    </tr>
    </thead>
    <tbody>""")
    for truth, l in table:
        print("<tr>")
        print(f'<td><img src="{truth}.png" style="margin: 1px 0px"></td>')
        highest_prob = l.argmax()
        for j, m in enumerate(l):
            if j == highest_prob:
                if highest_prob == truth:
                    colour = "green"
                else:
                    colour = "red"
                print(f'<td style="color:{colour};">{int(round(m*100))}%</td>')
            else:
                print(f"<td>{int(round(m*100))}%</td>")
        print("</tr>")
    print("""</tbody>
    </table>""")

#print_table(table)